In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.expand_frame_repr', False)  # keine Umbrüche mehr
pd.set_option('display.max_columns', None)         # alle Spalten anzeigen

In [ ]:
#example prices for a 4s frequency
def create_price_df():
    idx = pd.date_range(
        start='2022-01-01 00:00:00',
        end='2024-12-31 23:59:56',
        freq='4S',
        tz='Europe/Berlin'
    )
    
    prices_neg = -np.abs(np.random.randn(len(idx))) -4                    
    prices_pos = np.abs(np.random.randn(len(idx))) + 4 
    
    df = pd.DataFrame({
        'prices_neg': prices_neg,
        'prices_pos': prices_pos
    }, index=idx)
    
    return df

df = create_price_df()
print(df)
df.to_pickle('srl_example_prices_4s.pkl')

In [ ]:
# join prices with setpoints

df_srl_prices = pd.read_pickle('srl_example_prices_4s.pkl')
df_srl_power_setpoint = pd.read_pickle('srl_soll_leistung_2022-2025_4s.pkl')

print(len(df_srl_prices))
print(len(df_srl_power_setpoint))

# join
df_srl_work = df_srl_prices.join(df_srl_power_setpoint, how='inner')
print(len(df_srl_work))
print(df_srl_work)
df_srl_work.to_pickle('srl_work_4s.pkl')

In [ ]:
df_srl_work = pd.read_pickle('srl_work_4s.pkl')

In [ ]:
# testen ob proftabel and set target
SPECIFIC_AGING_COSTS = 4.7  # €/MWh
SYSTEM_POWER = 1           # MW
MWH_USED_PER_P = SYSTEM_POWER * 4/3600   # MWh pro 4 s

df = df_srl_work.copy()

# initialize work targets
df['work_target_neg'] = 0.0 
df['work_target_pos'] = 0.0

mask_neg = (
    (-df['prices_neg'] > SPECIFIC_AGING_COSTS)
    & (df['neg_setpoint'] < 0)
)
df.loc[mask_neg, 'work_target_neg'] = MWH_USED_PER_P

mask_pos = (
    (df['prices_pos'] >  SPECIFIC_AGING_COSTS)
    & (df['pos_setpoint'] > 0)
)
df.loc[mask_pos, 'work_target_pos'] = MWH_USED_PER_P

print(df)

In [ ]:
import numpy as np
import numba as nb

# Benchmark-Parameter
MWH_USED_PER_P      = MWH_USED_PER_P    # MWh pro 4 s pro MW
RAMP_TIME_S         = 5*60    # 300 s
RAMP_RATE_MWH_PER_S = MWH_USED_PER_P / RAMP_TIME_S
RAMP_STEP_MWH       = RAMP_RATE_MWH_PER_S * 4  # MWh pro 4 s





In [ ]:
@nb.njit
def compute_ramp(target: np.ndarray, ramp_step: float) -> np.ndarray:
    n      = target.shape[0]
    actual = np.empty(n, dtype=np.float64)
    prev   = 0.0
    for i in range(n):
        delta = target[i] - prev
        if   delta >  ramp_step:
            prev += ramp_step
        elif delta < -ramp_step:
            prev -= ramp_step
        else:
            prev  = target[i]
        actual[i] = prev
    return actual

# Anwendung
target = df['work_target_pos'].to_numpy()
# Erstes Mal JIT-Kompilierung dauert etwas, danach
actual = compute_ramp(target, RAMP_STEP_MWH)
df['actual_mwh_pos'] = actual


In [ ]:
@nb.njit
def compute_ramp(target: np.ndarray, ramp_step: float) -> np.ndarray:
    n      = target.shape[0]
    actual = np.empty(n, dtype=np.float64)
    prev   = 0.0
    for i in range(n):
        delta = target[i] - prev
        if   delta >  ramp_step:
            prev += ramp_step
        elif delta < -ramp_step:
            prev -= ramp_step
        else:
            prev  = target[i]
        actual[i] = prev
    return actual

# Anwendung
target = df['work_target_neg'].to_numpy()
# Erstes Mal JIT-Kompilierung dauert etwas, danach
actual = compute_ramp(target, RAMP_STEP_MWH)
df['actual_mwh_neg'] = actual


In [ ]:
print(df)

In [99]:
df_period = df.copy()
print(df_period)
# filter to 2022-06
df_period = df_period.loc['2022-06-01 00:00:00':'2022-06-30 23:59:56']
#to csv
df_period.to_csv('srl_work_2022.csv', index=True)

                           prices_neg  prices_pos  GERMANY_aFRR_SETPOINT_[MW]  neg_setpoint  pos_setpoint  work_target_neg  work_target_pos  actual_mwh_pos  actual_mwh_neg
2022-01-01 00:00:00+01:00   -4.742437    5.302215                  -61.080002    -61.080002           0.0         0.001111              0.0             0.0        0.000015
2022-01-01 00:00:04+01:00   -5.568450    4.854315                  -29.367001    -29.367001           0.0         0.001111              0.0             0.0        0.000030
2022-01-01 00:00:08+01:00   -4.667707    5.482987                  -27.200001    -27.200001           0.0         0.000000              0.0             0.0        0.000015
2022-01-01 00:00:12+01:00   -5.745297    4.768736                  -50.042000    -50.042000           0.0         0.001111              0.0             0.0        0.000030
2022-01-01 00:00:16+01:00   -4.692246    4.568923                  -22.200001    -22.200001           0.0         0.000000              0.0 